# Commodity Trading with Regret

### Background: Evans' Example 4.6.2
In **Chapter 4, Example 2 (Commodity Trading)**, Evans introduces a model for optimal trading. The model relies on the assumption that "the price of the wheat is known for the entire trading period". This is extremely unrealistic. Nonetheless, this model provides a baseline, "how well could have I performed had I known the future?". It thus allows us to measure the regret of a given strategy.

---

### The Goal: Back-Testing with a Theoretical Ceiling
We propose a **Regret Minimization** challenge.
Suppose we trade a highly cyclical sector (e.g., **XLE**, the Energy Sector). Theoretically, a Machine Learning model should be able to learn the "Buy Low, Sell High" trends. By using the Pontryagin Maximum Principle (PMP), we can compute the **absolute best possible performance** for any given price history and use this to train our model.

We measure our loss involving the regret factor:
$$\text{Regret} = (\text{Theoretical Best via PMP}) - (\text{Neural Network Performance})$$

---

### The Pipeline
We will implement the following 5-step workflow:

1.  **Extract Physics:** Download **XLE** (Energy ETF) data. Perform a cyclical curve fit to extract trend, frequency ($\omega$), and amplitude ($A$) statistics.
2.  **The Multiverse:** Generate "Alternate Universes" (synthetic sample paths) based on the statistics extracted in Step 1.
3.  **The Oracle (PMP):** Compute the "absolute best" trading strategy for each alternate universe using Evans' PMP solution (since we know the full history in the simulation).
4.  **The Student (ML):** Instantiate a Neural Network (e.g., LSTM) to learn the trading strategy. The model is trained to minimize the **regret** (imitate the Oracle).
5.  **The Final Exam:** Back-test the trained model on **Real XLE Data** to see if it captured the underlying physics of the market.

---

**Who will come out on top?**

In [ ]:
# Step 1: Imports and Data Extraction

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import warnings, os, time
warnings.filterwarnings('ignore')

SAVE_DIR = '/Users/yechanjeong/Desktop/stock'
np.random.seed(42)
torch.manual_seed(42)

# Configuration
ASSET_TICKER = 'XLE'
HOLDING_COST = 0.0005   # Daily holding cost
MAX_RATE = 5.0           # Max shares to buy/sell per day
MAX_INV = 80.0           # Max inventory position
WINDOW_SIZE = 30         # Lookback window for the Neural Network
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {DEVICE}")

# --- A. Data Ingestion ---
print("Fetching Market Data...")
train_data_raw = yf.download(ASSET_TICKER, start='1999-01-01', end='2024-01-01', progress=False)
eval_data_raw = yf.download(ASSET_TICKER, start='2024-01-01', end='2026-02-20', progress=False)

# Save for reproducibility
train_data_raw.to_csv(os.path.join(SAVE_DIR, 'xle_train.csv'))
eval_data_raw.to_csv(os.path.join(SAVE_DIR, 'xle_eval.csv'))

# Reload with consistent format
train_data = pd.read_csv(os.path.join(SAVE_DIR, 'xle_train.csv'), header=[0,1], index_col=0, parse_dates=True)
eval_data = pd.read_csv(os.path.join(SAVE_DIR, 'xle_eval.csv'), header=[0,1], index_col=0, parse_dates=True)

train_prices = train_data['Close'][ASSET_TICKER].values.astype(float)
eval_prices = eval_data['Close'][ASSET_TICKER].values.astype(float)
eval_dates = eval_data['Close'][ASSET_TICKER].index
train_dates = train_data['Close'][ASSET_TICKER].index

print(f"Training: {len(train_prices)} days, Eval: {len(eval_prices)} days")

# --- B. The Physics Fit ---
log_prices = np.log(train_prices)
t_values = np.arange(len(log_prices))

def log_cyclical_model(t, drift, intercept, amp, omega, phase):
    return drift * t + intercept + amp * np.sin(omega * t + phase)

guess_omega = 2 * np.pi / (252 * 7)
p0 = [(log_prices[-1]-log_prices[0])/len(log_prices), log_prices[0], np.std(log_prices), guess_omega, 0]
params, _ = curve_fit(log_cyclical_model, t_values, log_prices, p0=p0)
drift, intercept, amp, omega, phase = params

fitted_curve = log_cyclical_model(t_values, *params)
residuals = log_prices - fitted_curve
noise_std = np.std(residuals)
daily_innov_std = np.std(np.diff(residuals))

print(f"\n--- Market Physics Extracted ---")
print(f"Annual Drift: {(np.exp(drift*252)-1)*100:.2f}%")
print(f"Cycle Amplitude: {amp*100:.2f}% | Period: {(2*np.pi/omega)/252:.2f} Years")
print(f"Noise Volatility: {noise_std*100:.2f}%")

# Visualization
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(train_dates, train_prices, label='XLE Training Data', alpha=0.6)
ax.plot(train_dates, np.exp(fitted_curve), 'r--', label='Fitted Geometric Cycle', linewidth=2)
ax.set_yscale('log')
ax.set_title(f"{ASSET_TICKER} (1999-2023) vs. Fitted Geometric Cycle")
ax.legend(); ax.grid(True, alpha=0.3); ax.set_xlabel('Date'); ax.set_ylabel('Price ($)')
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig1_physics_fit.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Step 2: The Multiverse (Synthetic + Real Path Generation)

CHUNK_LENGTH = 300  # ~1.2 years of trading days
NUM_SYNTH = 200
STRIDE = 30  # Dense overlap for more real data coverage

def generate_synthetic_path(n_days):
    """Generate a synthetic price path with randomized physics parameters."""
    d = drift * np.random.uniform(0.3, 2.0)
    a = amp * np.random.uniform(0.3, 3.0)
    w = omega * np.random.uniform(0.5, 2.0)
    ph = np.random.uniform(0, 2*np.pi)
    sp = np.random.uniform(15, 150)
    t = np.arange(n_days)
    inn = np.random.normal(0, daily_innov_std * np.random.uniform(0.3, 2.0), n_days)
    noise = np.cumsum(inn)
    lp = np.log(sp) + d*t + a*np.sin(w*t+ph) + noise
    lp = np.clip(lp, np.log(3), np.log(600))
    return np.exp(lp)

synth_paths = [generate_synthetic_path(CHUNK_LENGTH) for _ in range(NUM_SYNTH)]

# Sliding windows from REAL training data
real_chunks = []
for start in range(0, len(train_prices) - CHUNK_LENGTH, STRIDE):
    real_chunks.append(train_prices[start:start+CHUNK_LENGTH])

# Weight real data 2x by including duplicates
all_paths = real_chunks + real_chunks + synth_paths
n_real = len(real_chunks)
print(f"Real chunks: {n_real} (x2={n_real*2}), Synthetic: {NUM_SYNTH}, Total: {len(all_paths)}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for i in range(min(30, NUM_SYNTH)):
    axes[0].plot(synth_paths[i], alpha=0.1, color='blue')
for i in range(min(20, n_real)):
    axes[0].plot(real_chunks[i], alpha=0.2, color='red')
axes[0].set_title('Synthetic (blue) & Real Chunks (red)')
axes[0].set_xlabel('Day'); axes[0].set_ylabel('Price ($)'); axes[0].grid(True, alpha=0.3)

real_ret = np.diff(np.log(train_prices))
synth_ret = np.diff(np.log(synth_paths[0]))
axes[1].hist(real_ret, bins=80, alpha=0.5, density=True, label='Real XLE')
axes[1].hist(synth_ret, bins=80, alpha=0.5, density=True, label='Synthetic')
axes[1].set_title('Return Distribution: Real vs Synthetic')
axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig2_synthetic_paths.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Step 3: The Oracle (PMP - Pontryagin Maximum Principle)

def pmp_oracle_full(prices, max_rate=MAX_RATE, holding_cost=HOLDING_COST, max_inv=MAX_INV):
    """
    Compute the PMP-optimal trading strategy for a known price path.
    
    Evans' Model: costate lambda(t) = lambda_0 + h*t
    Bang-bang control: u(t) = max_rate * sign(lambda(t) - p(t))
    Binary search on lambda_0 to satisfy terminal constraint x(T) ~ 0.
    
    Returns: pnl, actions, switching_function (oracle's unclipped intent)
    """
    T = len(prices)
    def simulate(lam0):
        x = 0.0; cash = 0.0; acts = np.zeros(T)
        for t in range(T):
            sigma = lam0 + holding_cost*t - prices[t]
            u = max_rate if sigma > 0 else -max_rate
            if x + u > max_inv: u = max(max_inv - x, 0)
            if x + u < -max_inv: u = min(-max_inv - x, 0)
            acts[t] = u; cash -= prices[t]*u + holding_cost*abs(x); x += u
        return x, cash + prices[-1]*x, acts

    lo, hi = float(prices.min()-200), float(prices.max()+200)
    for _ in range(200):
        mid = (lo+hi)/2; fx, _, _ = simulate(mid)
        if fx > 0.5: hi = mid
        elif fx < -0.5: lo = mid
        else: break
    lam0 = (lo+hi)/2
    _, pnl, acts = simulate(lam0)
    # Switching function: oracle's INTENT before inventory clipping
    sf = np.array([lam0 + holding_cost*t - prices[t] for t in range(T)])
    return pnl, acts, sf

# Compute oracle for all paths (cache duplicates)
print(f"Computing PMP oracle for {len(all_paths)} paths...")
all_oracle_pnls, all_oracle_acts, all_oracle_sfs = [], [], []

for i, path in enumerate(all_paths):
    if i >= n_real and i < 2*n_real:  # Second copy of real chunks - reuse
        orig_i = i - n_real
        all_oracle_pnls.append(all_oracle_pnls[orig_i])
        all_oracle_acts.append(all_oracle_acts[orig_i])
        all_oracle_sfs.append(all_oracle_sfs[orig_i])
    else:
        pnl, acts, sf = pmp_oracle_full(path)
        all_oracle_pnls.append(pnl); all_oracle_acts.append(acts); all_oracle_sfs.append(sf)
    if (i+1) % 100 == 0: print(f"  {i+1}/{len(all_paths)}")

oracle_pnls = np.array(all_oracle_pnls)
print(f"Oracle PnL: mean=${np.mean(oracle_pnls):.0f}")

# Visualization: Oracle on a real data chunk
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
idx = n_real // 2
axes[0].plot(all_paths[idx], 'b-')
axes[0].set_title(f'Real Chunk #{idx} (Oracle PnL=${oracle_pnls[idx]:.0f})')
axes[0].set_ylabel('Price ($)'); axes[0].grid(True, alpha=0.3)

sf = all_oracle_sfs[idx]
axes[1].plot(sf, 'purple', alpha=0.7)
axes[1].fill_between(range(len(sf)), sf, alpha=0.2, color='purple')
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
axes[1].set_title('Switching Function sigma(t) = lambda_0 + h*t - p(t)  [>0 = buy intent, <0 = sell]')
axes[1].set_ylabel('sigma(t)'); axes[1].grid(True, alpha=0.3)

axes[2].bar(range(CHUNK_LENGTH), all_oracle_acts[idx],
            color=['green' if a>0 else 'red' for a in all_oracle_acts[idx]], alpha=0.6, width=1)
axes[2].set_title('Oracle Actions (inventory-clipped bang-bang)')
axes[2].set_ylabel('Action (u)'); axes[2].grid(True, alpha=0.3)

inv = np.cumsum(all_oracle_acts[idx])
axes[3].fill_between(range(CHUNK_LENGTH), inv, alpha=0.3, color='orange')
axes[3].plot(inv, 'orange'); axes[3].set_title('Oracle Inventory')
axes[3].set_ylabel('Inventory'); axes[3].set_xlabel('Trading Day'); axes[3].grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig3_oracle_example.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Step 4: The Student (GRU Ensemble Neural Network)

class TradingGRU(nn.Module):
    """2-layer GRU with MLP head, outputs trading signal in [-1, 1]."""
    def __init__(self, input_size=5, hidden_size=96, num_layers=2, dropout=0.15):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers,
                         batch_first=True, dropout=dropout)
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 48), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(48, 1), nn.Tanh()
        )
    def forward(self, x):
        out, _ = self.gru(x)
        return self.head(out[:, -1, :])

def create_features(prices, window_size):
    """Create 5 features per timestep: z-score, returns, vol, mean-rev, momentum."""
    n = len(prices)
    log_p = np.log(np.clip(prices, 1e-6, None)).astype(np.float32)
    returns = np.zeros(n, dtype=np.float32); returns[1:] = np.diff(log_p)
    sma = np.zeros(n, dtype=np.float32)
    for i in range(n): sma[i] = np.mean(log_p[max(0,i-50):i+1])
    vol = np.zeros(n, dtype=np.float32)
    for i in range(1, n): vol[i] = np.std(returns[max(0,i-20):i+1])
    vol[0] = vol[1] if n > 1 else 0.01

    features_list = []
    for i in range(window_size, n):
        w = log_p[i-window_size:i]; p_std = max(np.std(w), 1e-6)
        norm_p = (w - np.mean(w)) / p_std
        ret_std = max(np.std(returns[max(0,i-100):i+1]), 1e-6)
        w_ret = returns[i-window_size:i] / ret_std
        vol_mean = max(np.mean(vol[max(0,i-100):i+1]), 1e-6)
        w_vol = vol[i-window_size:i] / vol_mean
        w_mr = (log_p[i-window_size:i] - sma[i-window_size:i]) / p_std
        w_mom = np.zeros(window_size, dtype=np.float32)
        for j in range(window_size):
            ti = i - window_size + j; lb = min(20, ti)
            w_mom[j] = (log_p[ti] - log_p[ti-lb]) / p_std if lb > 0 else 0
        features_list.append(np.stack([norm_p, w_ret, w_vol, w_mr, w_mom], axis=-1).astype(np.float32))
    result = np.array(features_list, dtype=np.float32)
    return np.clip(np.nan_to_num(result, nan=0.0, posinf=5.0, neginf=-5.0), -10.0, 10.0)

def create_training_data():
    """Target: tanh of normalized switching function (oracle's intent)."""
    all_X, all_y = [], []
    for i in range(len(all_paths)):
        feats = create_features(all_paths[i], WINDOW_SIZE)
        sf = all_oracle_sfs[i][WINDOW_SIZE:]
        price_scale = np.std(all_paths[i]) * 0.5
        target = np.tanh(sf / max(price_scale, 1.0)).astype(np.float32)
        ml = min(len(feats), len(target))
        all_X.append(feats[:ml]); all_y.append(target[:ml])
    X = np.concatenate(all_X); y = np.concatenate(all_y)
    mask = np.isfinite(X).all(axis=(1,2)) & np.isfinite(y)
    return X[mask], y[mask]

print("Building training data from switching function targets...")
X_train, y_train = create_training_data()
print(f"Samples: {len(X_train)}, target: mean={y_train.mean():.3f}, std={y_train.std():.3f}")
print(f"Positive (buy): {(y_train>0).mean()*100:.1f}%, Negative (sell): {(y_train<0).mean()*100:.1f}%")

X_tensor = torch.FloatTensor(X_train)
y_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
dataset = TensorDataset(X_tensor, y_tensor)

# --- Train Ensemble of 3 GRU Models ---
NUM_MODELS = 3
NUM_EPOCHS = 80
models = []

for m_idx in range(NUM_MODELS):
    print(f"\n--- Training model {m_idx+1}/{NUM_MODELS} ---")
    torch.manual_seed(42 + m_idx * 100)
    dataloader = DataLoader(dataset, batch_size=512, shuffle=True)
    model = TradingGRU(input_size=5, hidden_size=96, num_layers=2, dropout=0.15)
    optimizer = optim.Adam(model.parameters(), lr=0.002)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min=1e-5)
    best_loss = float('inf'); losses = []

    for epoch in range(NUM_EPOCHS):
        model.train(); el = 0.0; nb = 0
        for xb, yb in dataloader:
            optimizer.zero_grad(); pred = model(xb)
            loss = nn.MSELoss()(pred, yb)
            if torch.isnan(loss): continue
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step(); el += loss.item(); nb += 1
        scheduler.step(); avg = el / max(nb, 1); losses.append(avg)
        if avg < best_loss:
            best_loss = avg
            best_state = {k: v.clone() for k, v in model.state_dict().items()}
        if (epoch+1) % 20 == 0:
            model.eval()
            with torch.no_grad():
                sp = model(X_tensor[:5000]).squeeze()
                acc = ((sp * y_tensor[:5000].squeeze()) > 0).float().mean()
            model.train()
            print(f"  Epoch {epoch+1}, Loss: {avg:.6f}, Dir Acc: {acc:.3f}")

    model.load_state_dict(best_state); models.append(model)
    print(f"  Best loss: {best_loss:.6f}")

# Ensemble accuracy
for m in models: m.eval()
with torch.no_grad():
    preds = torch.stack([m(X_tensor[:5000]) for m in models]).mean(dim=0).squeeze()
    acc = ((preds * y_tensor[:5000].squeeze()) > 0).float().mean()
print(f"\nEnsemble directional accuracy: {acc:.3f}")

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(losses, 'b-', linewidth=2)
ax.set_title('Training Loss (MSE on Switching Function)'); ax.set_xlabel('Epoch'); ax.set_ylabel('Loss')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig4_training_loss.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Step 5: The Final Exam (Back-testing on Real 2024-2025 Data)

def backtest_ensemble(models, prices, window_size, max_rate, holding_cost, max_inv=MAX_INV):
    """Run ensemble model on price series with terminal constraint x(T)=0."""
    for m in models: m.eval()
    features = create_features(prices, window_size)
    n = len(prices); actions = np.zeros(n); inventory = np.zeros(n)
    x = 0.0; cash = 0.0

    # Reserve last ceil(max_inv/max_rate) days for forced liquidation (same as oracle's x(T)=0)
    liquidation_days = int(np.ceil(max_inv / max_rate))
    last_nn_day = len(features) - liquidation_days

    with torch.no_grad():
        for i in range(len(features)):
            t = i + window_size
            days_left = len(features) - i

            if i < last_nn_day:
                # Normal NN trading
                feat = torch.FloatTensor(features[i:i+1])
                raw = np.mean([m(feat).item() for m in models])
                u = np.sign(raw) * max_rate * min(abs(raw) * 1.5, 1.0)
            else:
                # Forced liquidation: unwind inventory to reach x(T)=0
                if abs(x) < 1e-6:
                    u = 0.0
                else:
                    u_needed = -x / max(days_left, 1)
                    u = np.clip(u_needed, -max_rate, max_rate)

            if x + u > max_inv: u = max(max_inv - x, 0)
            if x + u < -max_inv: u = min(-max_inv - x, 0)
            actions[t] = u; cash -= prices[t]*u + holding_cost*abs(x); x += u; inventory[t] = x

    cash += prices[-1] * x  # residual (should be ~0)
    return cash, actions, inventory

def pmp_oracle_simple(prices, max_rate=MAX_RATE, holding_cost=HOLDING_COST, max_inv=MAX_INV):
    """PMP oracle for evaluation (returns pnl, actions)."""
    T = len(prices)
    def simulate(lam0):
        x=0.0; cash=0.0; acts=np.zeros(T)
        for t in range(T):
            sigma = lam0 + holding_cost*t - prices[t]
            u = max_rate if sigma > 0 else -max_rate
            if x+u > max_inv: u = max(max_inv-x, 0)
            if x+u < -max_inv: u = min(-max_inv-x, 0)
            acts[t]=u; cash -= prices[t]*u + holding_cost*abs(x); x+=u
        return x, cash+prices[-1]*x, acts
    lo, hi = float(prices.min()-200), float(prices.max()+200)
    for _ in range(200):
        mid=(lo+hi)/2; fx,_,_=simulate(mid)
        if fx>0.5: hi=mid
        elif fx<-0.5: lo=mid
        else: break
    _,pnl,acts = simulate((lo+hi)/2)
    return pnl, acts

def compute_cum_pnl(prices, actions, holding_cost):
    cum = np.zeros(len(prices)); x=0.0; cash=0.0
    for t in range(len(prices)):
        cash -= prices[t]*actions[t] + holding_cost*abs(x)
        x += actions[t]; cum[t] = cash + x*prices[t]
    return cum

# --- Run the back-test ---
print(f"Back-testing on {len(eval_prices)} days of real XLE data...")
print(f"(Both Oracle and NN enforce terminal constraint x(T)=0)")
nn_pnl, nn_actions, nn_inv = backtest_ensemble(models, eval_prices, WINDOW_SIZE, MAX_RATE, HOLDING_COST)
oracle_pnl, oracle_acts = pmp_oracle_simple(eval_prices)
oracle_inv = np.cumsum(oracle_acts)
bh_pnl = eval_prices[-1] - eval_prices[0]

nn_cum = compute_cum_pnl(eval_prices, nn_actions, HOLDING_COST)
oracle_cum = compute_cum_pnl(eval_prices, oracle_acts, HOLDING_COST)
bh_cum = eval_prices - eval_prices[0]

regret = oracle_pnl - nn_pnl
ratio = nn_pnl / oracle_pnl * 100 if oracle_pnl > 0 else 0

print(f"\n{'='*55}")
print(f"  EVALUATION RESULTS (2024-2025)")
print(f"{'='*55}")
print(f"  PMP Oracle (Theoretical Best): ${oracle_pnl:>12.2f}")
print(f"  Neural Network (Ensemble):     ${nn_pnl:>12.2f}")
print(f"  Buy-and-Hold Benchmark:        ${bh_pnl:>12.2f}")
print(f"{'='*55}")
print(f"  Regret (Oracle - NN):          ${regret:>12.2f}")
print(f"  NN captures {ratio:.1f}% of Oracle")
print(f"  NN vs Buy-Hold:                ${nn_pnl - bh_pnl:>12.2f}")
print(f"{'='*55}")

# In-sample sanity check
is_prices = train_prices[-504:]
nn_is, _, _ = backtest_ensemble(models, is_prices, WINDOW_SIZE, MAX_RATE, HOLDING_COST)
or_is, _ = pmp_oracle_simple(is_prices)
print(f"\nIn-sample (last 2yr): NN=${nn_is:.2f} ({nn_is/or_is*100:.1f}% of Oracle=${or_is:.2f})")

# --- FIGURES ---
# Figure 5: Full evaluation
fig, axes = plt.subplots(4, 1, figsize=(14, 16))
axes[0].plot(eval_dates, eval_prices, 'k-', lw=1.5, label='XLE Price')
axes[0].set_title('XLE Evaluation Period (2024-2025)', fontsize=14)
axes[0].set_ylabel('Price ($)'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(eval_dates, nn_cum, 'b-', lw=2, label=f'NN Ensemble (${nn_pnl:.2f})')
axes[1].plot(eval_dates, oracle_cum, 'r--', lw=2, label=f'PMP Oracle (${oracle_pnl:.2f})')
axes[1].plot(eval_dates, bh_cum, 'g:', lw=2, label=f'Buy & Hold (${bh_pnl:.2f})')
axes[1].set_title('Cumulative P&L Comparison', fontsize=14)
axes[1].set_ylabel('P&L ($)'); axes[1].legend(fontsize=11); axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', alpha=0.2)

axes[2].bar(range(len(nn_actions)), nn_actions,
            color=['green' if a>0 else ('red' if a<0 else 'gray') for a in nn_actions], alpha=0.5, width=1)
axes[2].set_title('NN Trading Actions', fontsize=14); axes[2].set_ylabel('Action (u)'); axes[2].grid(True, alpha=0.3)

axes[3].fill_between(range(len(nn_inv)), nn_inv, alpha=0.3, color='blue')
axes[3].plot(nn_inv, 'b-', lw=1)
axes[3].set_title('NN Inventory Position', fontsize=14); axes[3].set_ylabel('Inventory')
axes[3].set_xlabel('Trading Day'); axes[3].grid(True, alpha=0.3)
axes[3].axhline(y=0, color='k', ls='--', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig5_evaluation_backtest.png'), dpi=150, bbox_inches='tight')
plt.show()

# Figure 6: Oracle vs NN comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes[0,0].plot(eval_dates, oracle_acts, 'r-', alpha=0.6)
axes[0,0].set_title('Oracle Actions', fontsize=13); axes[0,0].set_ylabel('Action'); axes[0,0].grid(True, alpha=0.3)
axes[0,1].plot(eval_dates, nn_actions, 'b-', alpha=0.6)
axes[0,1].set_title('NN Actions', fontsize=13); axes[0,1].set_ylabel('Action'); axes[0,1].grid(True, alpha=0.3)
axes[1,0].fill_between(eval_dates, oracle_inv, alpha=0.2, color='red')
axes[1,0].plot(eval_dates, oracle_inv, 'r-', alpha=0.7)
axes[1,0].set_title('Oracle Inventory'); axes[1,0].set_ylabel('Inventory'); axes[1,0].grid(True, alpha=0.3)
axes[1,1].fill_between(eval_dates, nn_inv, alpha=0.2, color='blue')
axes[1,1].plot(eval_dates, nn_inv, 'b-', alpha=0.7)
axes[1,1].set_title('NN Inventory'); axes[1,1].set_ylabel('Inventory'); axes[1,1].grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig6_actions_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

# Figure 7: Performance bar chart
fig, ax = plt.subplots(figsize=(9, 6))
strats = ['PMP Oracle\n(Theoretical Best)', 'NN Ensemble\n(Our Model)', 'Buy & Hold\n(Benchmark)']
pvals = [oracle_pnl, nn_pnl, bh_pnl]
bars = ax.bar(strats, pvals, color=['gold', 'steelblue', 'lightcoral'], edgecolor='black', lw=1.2)
mx = max(abs(p) for p in pvals) if any(p!=0 for p in pvals) else 1
for b, p in zip(bars, pvals):
    off = mx*0.03*(1 if p>=0 else -1)
    ax.text(b.get_x()+b.get_width()/2., b.get_height()+off,
            f'${p:.2f}', ha='center', va='bottom' if p>=0 else 'top', fontsize=12, fontweight='bold')
ax.set_title('Strategy Performance Comparison (2024-2025)', fontsize=14)
ax.set_ylabel('Total P&L ($)'); ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0, color='k', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'fig7_performance_summary.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\nAll figures saved to:", SAVE_DIR)